In [2]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [3]:
%%appyter hide_code

{% do SectionField(
    name='primary',
    title='Upload Tumor Expression',
    img='upload.png'
) %}

{% set data_type = TabField(
    name='data_type',
    label='Data Type',
    default='scRNA-seq',
    description='Start with either scRNA-seq or bulk RNA-seq data. If bulk RNA-seq data is selected we will utilize pre-curated reference matrices to identify cell type-specific expression vectors.',
    required=True,
    section='primary',
    choices={
        'scRNA-seq': [
            TabField(
            name='sc_data_type',
            label='Data Format (Control Profile)',
            default='Plain Text',
            description='Choose a format for scRNA-seq data',
            required=True,
            choices= {
                'Plain Text': [
                        CustomFileField(
                        name='sc_rna_file_ctrl',
                        label='control scRNA-seq',
                        description='''
    File should be a tsv/csv of the form:

                                        cell 1   cell 2    ...
                    ------------------------------
            Gene/Protein 1    0          2       ...
                    ------------------------------
            Gene/Protein 2    1         1        ...
                    ------------------------------
                        ...                       ... 
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    CustomFileField(
                        name='sc_meta_file',
                        label='metadata',
                        description='''
                        File should be a tsv/csv of the form:

                            Patient 1 Tumor    Patient 2 Tumor  ...
                        ---------------------------------------
                        Gene/Protein 1    0                   200       ...
                        ---------------------------------------
                        Gene/Protein 2    5                   180       ...
                        ---------------------------------------
                        ...                       ...                    ...        ...
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    TextField(
                        name='cell_type_col',
                        label='Cell Type Column (Optional)',
                        description='Name of cell type column in uploaded metadata (if not selected, cell type identification will be performed)',
                        default='cell_type',
                        required=False
                    )
                    ],
                '.mtx': [
                        CustomFileField(
                        name='sc_rna_file',
                        label='scRNA-seq',
                        description='''
    File should be a tsv/csv of the form:

                                        cell 1   cell 2    ...
                    ------------------------------
            Gene/Protein 1    0          2       ...
                    ------------------------------
            Gene/Protein 2    1         1        ...
                    ------------------------------
                        ...                       ... 
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    CustomFileField(
                        name='sc_meta_file',
                        label='metadata',
                        description='''
                        File should be a tsv/csv of the form:

                            Patient 1 Tumor    Patient 2 Tumor  ...
                        ---------------------------------------
                        Gene/Protein 1    0                   200       ...
                        ---------------------------------------
                        Gene/Protein 2    5                   180       ...
                        ---------------------------------------
                        ...                       ...                    ...        ...
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    )]
            }
        ),
        TabField(
            name='sc_data_type',
            label='Data format',
            default='Plain Text',
            description='Choose a format for scRNA-seq data',
            required=True,
            choices= {
                'Plain Text': [
                        CustomFileField(
                        name='sc_rna_file',
                        label='scRNA-seq',
                        description='''
    File should be a tsv/csv of the form:

                                        cell 1   cell 2    ...
                    ------------------------------
            Gene/Protein 1    0          2       ...
                    ------------------------------
            Gene/Protein 2    1         1        ...
                    ------------------------------
                        ...                       ... 
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    CustomFileField(
                        name='sc_meta_file',
                        label='metadata',
                        description='''
                        File should be a tsv/csv of the form:

                            Patient 1 Tumor    Patient 2 Tumor  ...
                        ---------------------------------------
                        Gene/Protein 1    0                   200       ...
                        ---------------------------------------
                        Gene/Protein 2    5                   180       ...
                        ---------------------------------------
                        ...                       ...                    ...        ...
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    TextField(
                        name='condition name',
                        label='condition name',
                        description='Name of the condition',
                        default='Tumor',
                        required=True
                    )
                    ],
                '.mtx': [
                        CustomFileField(
                        name='sc_rna_file',
                        label='scRNA-seq',
                        description='''
    File should be a tsv/csv of the form:

                                        cell 1   cell 2    ...
                    ------------------------------
            Gene/Protein 1    0          2       ...
                    ------------------------------
            Gene/Protein 2    1         1        ...
                    ------------------------------
                        ...                       ... 
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    CustomFileField(
                        name='sc_meta_file',
                        label='metadata',
                        description='''
                        File should be a tsv/csv of the form:

                            Patient 1 Tumor    Patient 2 Tumor  ...
                        ---------------------------------------
                        Gene/Protein 1    0                   200       ...
                        ---------------------------------------
                        Gene/Protein 2    5                   180       ...
                        ---------------------------------------
                        ...                       ...                    ...        ...
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    )]
            }
        )],
        'bulk RNA-seq':[
            CustomFileField(
                name=' bulk_expr',
                label='RNA-seq expression',
                description='''
                File should be a tsv/csv of the form:

                    Patient 1 Tumor    Patient 2 Tumor  ...
                ---------------------------------------
                Gene/Protein 1    0                   200       ...
                ---------------------------------------
                Gene/Protein 2    5                   180       ...
                ---------------------------------------
                ...                       ...                    ...        ...
                ''',
                default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                required=False,
                examples={
                    'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                },
                section='primary',
            ),
            CustomFileField(
                        name='sc_meta_file',
                        label='metadata',
                        description='''
                        File should be a tsv/csv of the form:

                            Patient 1 Tumor    Patient 2 Tumor  ...
                        ---------------------------------------
                        Gene/Protein 1    0                   200       ...
                        ---------------------------------------
                        Gene/Protein 2    5                   180       ...
                        ---------------------------------------
                        ...                       ...                    ...        ...
                        ''',
                        default='data/sarcoma-v2-proteome-SpectrumMill-ratio-QCfilter-NArm.tsv',
                        required=False,
                        examples={
                            'BR_proteomics_tumor.tsv': 'https://s3.amazonaws.com/multiomics2paper/public/BR_proteomics_tumor.tsv',
                        },
                    ),
                    TextField(
                        name='condition_col',
                        label='Condition Column',
                        description='Name of the condition column in uploaded metadata',
                        default='Condition',
                        required=True
                    ),
                    TextField(
                        name='ctrl_condition',
                        label='Control Condition Name',
                        description='Name of the control condition in the condition column in uploaded metadata',
                        default='healthy',
                        required=True
                    ),
                    ChoiceField(
                        name='reference',
                        label='Single Cell Reference',
                        description='Choose a single cell reference to use for cell type deconvolution',
                        choices={
                            "Pancreas - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Pancreas - Tritschler et al. (Mouse)": "https://cellxgene.cziscience.com/collections/0a77d4c0-d5d0-40f0-aa1a-5e1429bcbd7e",
                            "Adipose Tissue - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Adipose Tissue - Emont et al. (Mouse)": "https://cellxgene.cziscience.com/collections/fe0e718d-2ee9-42cc-894b-0b490f437dfd",
                            "Adipose Tissue - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Liver - MacParland et al. 2018 (Human)": "https://cellxgene.cziscience.com/collections/bd5230f4-cd76-4d35-9ee5-89b3e7475659",
                            "Blood - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Kidney - Tabula Sapiens(Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Brain - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Skeletal Muscle - Domínguez Conde et al. 2022 (Human)": "https://cellxgene.cziscience.com/collections/62ef75e4-cbea-454e-a0ce-998ec40223d3",
                            "Heart - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Heart - Litviňuková et al. 2020 (Mouse)": "https://cellxgene.cziscience.com/collections/b52eb423-5d0d-4645-b217-e1c6d38b2e72",
                            "Heart - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Muscle - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Muscle - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Skin - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Retina - Li et al. 2023 (Human)": "https://cellxgene.cziscience.com/collections/4c6eaf5c-6d57-4c76-b1e9-60df8c655f1e",
                            "Retina - Cowan et al. 2020 (Mouse)": "https://cellxgene.cziscience.com/collections/2f4c738f-e2f3-4553-9db2-0582a38ea4dc",
                            "Breast - Reed et al. 2024 (Human)": "https://cellxgene.cziscience.com/collections/48259aa8-f168-4bf5-b797-af8e88da6637",
                            "Lung - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5",
                            "Lung - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Uterus - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Bladder - Tabula Sapiens (Human)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Bladder - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb",
                            "Gingiva - Easter et al. 2024 (Human)": "https://cellxgene.cziscience.com/collections/71f4bccf-53d4-4c12-9e80-e73bfb89e398",
                            "Gingiva - Easter et al. 2024 (Mouse)": "https://cellxgene.cziscience.com/collections/67ba665e-0611-4b53-a522-40c2e0dc6df7",
                            "Spleen - Tabula Muris (Mouse)": "https://cellxgene.cziscience.com/collections/0b9d8a04-bb9d-44da-aa27-705bb65b54eb"
                            }
                    )
         ]
        }
) %}


{% set n_neighbors = IntField(
    name='n_neighbors',
    label='N neighbors', 
    default=15, 
    min=2,
    max=100,
    step=1,
    description='The size of local neighborhood (in terms of number of neighboring data points) used for manifold approximation.', 
    section='primary'
)
%}

{% set min_dist = FloatField(
    name='min_dist',
    label='Min Distance', 
    default=0.01, 
    min=0.00001,
    max=0.9,
    step=0.00001,
    description='The effective minimum distance between embedded points.', 
    section='primary'
)
%}

{% set resolution = FloatField(
    name='resolution',
    label='Resolution', 
    default=1, 
    min=0.1,
    max=2,
    step=0.1,
    description='A parameter value controlling the coarseness of the clustering. Higher values lead to more clusters.', 
    section='primary'
)
%}


{% set membrane_screener_list = 'https://appyters.maayanlab.cloud/storage/Tumor_Gene_Target_Screener/surfaceome.csv' %}


In [4]:
from helpers import *
import os
import re
import random
import qnorm
from string import ascii_uppercase
from tqdm import tqdm
import numpy as np
import pandas as pd
import scanpy as sc

from matplotlib import cm
from matplotlib.patches import Rectangle
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline

import networkx as nx

from IPython.display import HTML, display, Markdown, FileLink
from matplotlib.gridspec import GridSpec
from matplotlib_venn import venn2
from maayanlab_bioinformatics.normalization import zscore_normalize, log2_normalize
from scipy.stats import zscore
from scipy.stats import ttest_ind, ttest_1samp, ttest_ind_from_stats
from maayanlab_bioinformatics.dge import limma_voom_differential_expression
from maayanlab_bioinformatics.harmonization.ncbi_genes import ncbi_genes_lookup
from maayanlab_bioinformatics.api import enrichr_link_from_genes
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings("ignore")

os.makedirs('results', exist_ok=True)
os.makedirs('figures', exist_ok=True)
try:
    lookup = ncbi_genes_lookup(organism='Mammalia/Homo_sapiens')
except:
    import urllib.request, json 
    with urllib.request.urlopen("https://s3.amazonaws.com/multiomics2paper/public/ncbi_genes_disambiguated.json") as url:
        ncbi_genes_disambiguated = json.load(url)
        lookup = ncbi_genes_disambiguated.get
    
fig_counter = 2
table_counter = 1
letter_counter = 0
discussion_results = {}

import sys
import contextlib
@contextlib.contextmanager
def suppress_output(stdout=True, stderr=True, dest=os.devnull):
    ''' Usage:
    with suppress_output():
        print('hi')
    '''
    dev_null = open(dest, 'a')
    if stdout:
        _stdout = sys.stdout
        sys.stdout = dev_null
    if stderr:
        _stderr = sys.stderr
        sys.stderr = dev_null
    try:
        yield
    finally:
        if stdout:
            sys.stdout = _stdout
        if stderr:
            sys.stderr = _stderr

In [ ]:
if has_rna_file:
    cluster_dges = {}
    dge_table = {}
    for c in sorted(clusters):
        cluster_dges[c] = {}
        cluster_samples = list(cluster_df[cluster_df['cluster'] == c].index.values)
        other_samples = [s for s in rna_df.columns if s not in cluster_samples]
        if rna_df.min().min() < 0:
            rna_df_dge = rna_df + abs(rna_df.min().min())
        else:
            rna_df_dge = rna_df
        with suppress_output():
            dge = limma_voom_differential_expression(
                rna_df_dge[other_samples], rna_df_dge[cluster_samples],
                voom_design=True,
            )
        dge_filename = f'results/cluster_{c}_vs_rest_dge_limma.tsv'
        dge.sort_values(by="adj.P.Val", ascending=True).to_csv(dge_filename, sep='\t')
        sig_dge = dge[dge['adj.P.Val'] < 0.01]
        sig_up_gs = list(sig_dge[sig_dge['logFC'] > 0].index.values)
        sig_dn_gs = list(sig_dge[sig_dge['logFC'] < 0].index.values)
        if len(sig_up_gs) > 2000:
            sig_up_gs = sig_up_gs[:2000]
        elif len(sig_up_gs) < 50:
            sig_dge = dge[dge['adj.P.Val'] < 0.05]
            sig_up_gs = list(sig_dge[sig_dge['logFC'] > 0].index.values)
        if len(sig_dn_gs) > 2000:
            sig_dn_gs = sig_dn_gs[:2000]
        elif len(sig_dn_gs) < 50:
            sig_dge = dge[dge['adj.P.Val'] < 0.05]
            sig_dn_gs = list(sig_dge[sig_dge['logFC'] > 0].index.values)
        cluster_dges[c]['up'] = sig_up_gs
        cluster_dges[c]['down'] = sig_dn_gs
        dge_table[c] = {}
        dge_table[c]['up'] = len(cluster_dges[c]['up'])
        dge_table[c]['down'] = len(cluster_dges[c]['down'])
        display(FileLink(dge_filename, result_html_prefix=f'Differential expression for cluster {c}: '))

In [ ]:
%%appyter code_exec

if has_rna_file:    
    targets = {}
    bgs = {'GTEx': 'gtex-gene-stats.tsv', 'ARCHS4': 'archs4-gene-stats.tsv', 'TS': 'ts_10x_cell-ontology-class_donors_tissue-labels_v1.tsv'}
    def find_targets(rna_df, bg, targets):
        df_bg_stats = pd.read_csv(f"https://appyters.maayanlab.cloud/storage/Tumor_Gene_Target_Screener/{bgs[bg]}", sep='\t', index_col=[0,1])
        df_bg_genes = df_bg_stats.unstack().index.map(lambda idx: lookup(idx.partition('.')[0]))
        df_bg_stats = df_bg_stats.unstack().groupby(df_bg_genes, observed=True).median().stack()
        df_bg_expr = df_bg_stats.loc[(slice(None), ['25%', '50%', '75%']), :].unstack()
        common_index = list(set(rna_df.index) & set(df_bg_expr.index))
        expr_df = rna_df.loc[common_index, :]
        index_name = expr_df.index.name
        expr_df.reset_index(inplace=True)
        expr_df.drop_duplicates(subset=index_name, inplace=True)
        expr_df.set_index(index_name, inplace=True, drop=True)
        target_distribution = df_bg_expr.loc[common_index, :].median(axis=1)
        df_expr_norm = qnorm.quantile_normalize(expr_df.loc[common_index, :], target=target_distribution)
        df_bg_expr_norm = qnorm.quantile_normalize(df_bg_expr.loc[common_index, :], target=target_distribution)
        for cluster in tqdm(list(cluster_df['cluster'].unique())):
            if cluster not in targets: targets[cluster] = {}
            cluster_samples = list(cluster_df[cluster_df['cluster'] == cluster].index.values)
            with suppress_output():
                df_bg_expr_norm.columns = df_bg_expr_norm.columns.to_flat_index().map(lambda s: ', '.join(s))
                dge = limma_voom_differential_expression(
                    df_bg_expr_norm, df_expr_norm[cluster_samples],
                    voom_design=True,
                )
                targets[cluster][bg] = dge[(dge['adj.P.Val'] < 0.01) & (dge['t'] > 0)].sort_values('t', ascending=False).index.values 

In [ ]:
if has_rna_file: 
    print('Finding Targets using ARCHS4')
    find_targets(rna_df, 'ARCHS4', targets)

In [ ]:
if has_rna_file:    
    print('Finding Targets using GTEx')
    find_targets(rna_df, 'GTEx', targets)

In [ ]:
if has_rna_file:      
    print('Finding Targets using Tabula Sapiens')
    find_targets(rna_df, 'TS', targets)

In [ ]:
%%appyter code_eval

if has_rna_file:
    YlGnBu = cm.get_cmap('YlGnBu_r', 8)
    cmap = {"None":YlGnBu(0), "ARCHS4": YlGnBu(1), "GTEx":YlGnBu(2), "TS": YlGnBu(4), "ARCHS4-GTEx":YlGnBu(3),  "ARCHS4-TS": YlGnBu(5), "GTEx-TS": YlGnBu(6), "All": YlGnBu(7)}

    top_targets_n = 100
    target_list = []
    for cluster in targets:
        for bg in targets[cluster]:
            targets[cluster][bg] = list(filter(lambda x: not x.startswith('PCDH') ,targets[cluster][bg]))
    for cluster in targets:
        for bg in targets[cluster]:
            target_list.extend(list(filter(lambda g: g in membrane_proteins, targets[cluster][bg][:top_targets_n])))
    if len(set(target_list)) < 100:
        top_targets_n = 500
        for cluster in targets:
            for bg in targets[cluster]:
                target_list.extend(list(filter(lambda g: g in membrane_proteins, targets[cluster][bg][:top_targets_n])))
                target_list.extend(list(targets[cluster][bg][:top_targets_n]))
    
    data1, data2, data3 = [], [], []
    similarity = []
    target_list = list(set(target_list))

    for gene in target_list:
        a = [1 if gene in targets[c]['ARCHS4'][:top_targets_n] else 0 for c in clusters]
        g = [2 if gene in targets[c]['GTEx'][:top_targets_n] else 0 for c in clusters]
        l = [4 if gene in targets[c]['TS'][:top_targets_n] else 0 for c in clusters]
        data1.append(a)
        data2.append(g)
        data3.append(l)
        similarity.append(np.dot(np.dot(np.array(a), np.array(g)), np.array(l)))

    data = np.add(np.add(data1, data2), data3)

    membrane_target_mat = pd.DataFrame(data)
    membrane_target_mat.columns = [f"Cluster {c}" for c in clusters]
    membrane_target_mat.index = target_list

    membrane_target_mat['count'] = membrane_target_mat.sum(axis=1)
    membrane_target_mat = membrane_target_mat[membrane_target_mat['count'] >= 7]
    membrane_target_mat = membrane_target_mat.rename_axis('Membrane Target').sort_values(by = ['count', 'Membrane Target'], ascending = [False, True]).drop('count', axis=1)

    h = membrane_target_mat.shape[0]

    cluster_targets = {}
    for col in membrane_target_mat.columns:
        cluster_targets[col + ' Cell Surface Target'] = ','.join(list(membrane_target_mat[col][membrane_target_mat[col] >= 5].index.values))

    g = sns.clustermap(membrane_target_mat, figsize=(4,0.3*h+2*(h<15)), cmap=YlGnBu, cbar_pos=None, dendrogram_ratio=0.1-(h<40)*0.01*(h-30), row_cluster=False, xticklabels=True, yticklabels=True)
    g.ax_row_dendrogram.legend(handles=[Rectangle((0, 0), 0, 0, color=val, label=key) for key, val in cmap.items()],
                                    title='Background', loc='upper right')

    plt.show()

    g.savefig(f'figures/{rna_filename}_membrane_target_matrix.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_membrane_target_matrix.svg', dpi=300)
    membrane_target_mat.to_csv(f'results/{rna_filename}_membrane_target_mat.csv')
    display(Markdown(f"__Fig. {fig_counter}__ Top 100 consensus targets for each cluster with adjusted p-value < 0.01 compared to ARCHS4, GTEx, and Tabula Sapiens (TS) healthy backgrounds with limma voom."))
    display(FileLink(f'results/{rna_filename}_membrane_target_mat.csv', result_html_prefix='Download Membrane Target Matrix: '))
    display(FileLink(f'figures/{rna_filename}_membrane_target_matrix.png', result_html_prefix='Membrane Target Matrix PNG: '))
    display(FileLink(f'figures/{rna_filename}_membrane_target_matrix.svg', result_html_prefix='Membrane Target Matrix SVG: '))

In [ ]:
if has_rna_file:
    display(Markdown(f"Cell surface targets were identified for each cluster and are ordered by the number of healthy tissues and cell type background atlases that were used to identify the same target (Fig. {fig_counter})."))
    fig_counter += 1